In [1]:
import os
from os.path import join
import sys
import numpy as np 
import pandas as pd
import seaborn as sns

In [2]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


# Overall l/r bias
Test whether there is an overall left vs right bias in the cursor behavior. This is a control test for the left/right eye bias that we see in face stimuli

**Note**: "left" is relative to the viewer

In [3]:
# load the combined cursorTracking data for all subjs
df = pd.read_table('../../data/groupData/allSubjsRaw.tsv', sep='\t')

In [4]:
df.shape

(608261, 7)

### Select the stimuli to include in the analysis
Choose the non-face and non-face-like stimuli

In [5]:
# function to label the image categories
def labelStimCategory(row):
    if "_images707/simpleFood" in row['stim']:
        label = 'portrait'
    elif "_images707/art/stillLife_" in row['stim']:
        label = 'stillLife'
    else:
        label = "Na"
    
    return label

In [6]:
# add image category label to dataframe
df['stimCategory'] = df.apply(labelStimCategory, axis=1)

In [7]:
# make sure this worked
df['stimCategory'].unique()

array(['Na', 'stillLife', 'portrait'], dtype=object)

In [8]:
# drop the NAs
df = df[df['stimCategory'] != 'Na']
df.shape

(74169, 8)

In [9]:
# write this out as a txt file so you don't have to load the big file again
df.to_csv('overallBias_allSubjsRaw.tsv', sep='\t', index=False)

In [10]:
df.head()

,ts,x,y,subjID,stim,trialNum,AOI,stimCategory
293,51,88.0,157.5,27235,_images707/art/stillLife_16.jpg,2,NaN,stillLife
294,68,88.0,157.5,27235,_images707/art/stillLife_16.jpg,2,NaN,stillLife
295,85,88.0,157.5,27235,_images707/art/stillLife_16.jpg,2,NaN,stillLife
296,102,88.0,157.5,27235,_images707/art/stillLife_16.jpg,2,NaN,stillLife
297,119,88.0,157.5,27235,_images707/art/stillLife_16.jpg,2,NaN,stillLife


---

### Label each datapoint according to which half of the image it lies on
In order to know what half of the image each datapoint falls on, have to know the dimensions of each stim (all stims are 707x565, but they are either landscape or portrait oriented). Step one is to make a table of image dimensions for each stim

In [12]:
stims = np.unique(df.stim)
stims

array(['_images707/art/stillLife_10.jpg',
       '_images707/art/stillLife_11.jpg',
       '_images707/art/stillLife_15.jpg',
       '_images707/art/stillLife_16.jpg', '_images707/art/stillLife_3.jpg',
       '_images707/simpleFood/simpleFood_1.jpg',
       '_images707/simpleFood/simpleFood_10.jpg',
       '_images707/simpleFood/simpleFood_11.jpg',
       '_images707/simpleFood/simpleFood_12.jpg',
       '_images707/simpleFood/simpleFood_2.jpg',
       '_images707/simpleFood/simpleFood_3.jpg',
       '_images707/simpleFood/simpleFood_4.jpg',
       '_images707/simpleFood/simpleFood_5.jpg',
       '_images707/simpleFood/simpleFood_6.jpg',
       '_images707/simpleFood/simpleFood_7.jpg',
       '_images707/simpleFood/simpleFood_8.jpg',
       '_images707/simpleFood/simpleFood_9.jpg'], dtype=object)

In [13]:
import matplotlib.image as mpimg

In [20]:
stimDirRoot = '../../stimuli/'
widths = np.array([], dtype=int32)
heights = np.array([], dtype=int32)
for s in stims:
    # open image
    thisStim = mpimg.imread(join(stimDirRoot, s))
    
    heights = np.append(heights, int(thisStim.shape[0]))
    widths = np.append(widths, int(thisStim.shape[1]))

stimSize = pd.DataFrame({'stim':stims, 'widths':widths, 'heights':heights})
stimSize

,heights,stim,widths
0,566,_images707/art/stillLife_10.jpg,707
1,566,_images707/art/stillLife_11.jpg,707
2,707,_images707/art/stillLife_15.jpg,566
3,566,_images707/art/stillLife_16.jpg,707
4,566,_images707/art/stillLife_3.jpg,707
5,707,_images707/simpleFood/simpleFood_1.jpg,566
6,707,_images707/simpleFood/simpleFood_10.jpg,567
7,707,_images707/simpleFood/simpleFood_11.jpg,566
8,707,_images707/simpleFood/simpleFood_12.jpg,566
9,707,_images707/simpleFood/simpleFood_2.jpg,566
